This notebook was meant to be an EDA on the airb&b data published @ kaggle by user 'Airb&b', the data was taken from their Boston dataset.
The first part of the notebook focus on understanding and cleaning the database. Users that look to EDA the database are welcome to take my preprocessing work as a reasonable/suitable starting point.
<br><br>
Due to a tight work schedule its not concluded yet. However there are a few nice correlations established to price and rating.

In [ ]:
import pandas as pd
import glob
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
import warnings
from scipy.stats.stats import pearsonr

In [ ]:
glob.glob('../input/*.csv')

In [ ]:

data = {}

for i in glob.glob('../input/*.csv'):
    data['%s' %(i.replace('.csv','').replace('../input/',''))]=pd.read_csv('%s' %i)

In [ ]:
print ('number of DBs:','\t', len(data),'\n')
print ('DB name','\t','DB shape','\n')

for i in data:
    print (i,'\t',data[i].shape)

The database is composed of 3 bases. <br>
We will look into them by order of magnitude (smallest first)The database is composed of 3 bases. <br>
We will look into them by order of magnitude (smallest first)

In [ ]:
data['calendar'].info()

In [ ]:
def correction(x):
    if type(x)==str:
        x=x.replace('$','')
        x=x.replace(',','')
        x=float(x)
    return (x)

def to_binary(x):
    if x=='f':
        x=x.replace('f','0')
        x=bool(x)
    if x=='t':
        x=x.replace('t','0')
        x=bool(x)
    return (x)

data['calendar']['price']=data['calendar']['price'].map(lambda x: correction(x))
data['calendar']['date']=pd.to_datetime(data['calendar']['date'])
data['calendar']['available']=data['calendar']['available'].map(lambda x: to_binary(x))

In [ ]:
data['calendar'].info()

We have properly converted dtypes() and converted {'t','f'} columns to binary values. <br>
We have also converted USD strings into floats.

In [ ]:
print (data['calendar']['listing_id'].value_counts().head(), '\n')

# for some reason theres a single user that has a dupplicated amount of appearances. the code bellow removes him.

for i in data['calendar']['listing_id'].value_counts()[data['calendar']['listing_id'].value_counts()!=365].index:
    data['calendar']['listing_id']=data['calendar'][data['calendar']['listing_id']!=i]

In [ ]:
data['calendar'].head()

That is it with calendar. <br>
Now onto reviews.

In [ ]:
data['reviews'].info()

In [ ]:
data['reviews']['date']=pd.to_datetime(data['reviews']['date'])
data['reviews'].head()

Okay. now the big one. <br>
listing has 95 features. naturally we expect some of them to present issues. <br>
Lets dive into it. <br>
<br>
First we look at columns with singular values and columns containing urls.

In [ ]:
unique_value_columns=[]
url_columns=[]

for i in data['listings'].columns:
    
    if len((data['listings'][i]).unique())==1:
        print (i, (data['listings'][i]).unique())
        unique_value_columns=unique_value_columns+[i]
    if 'url' in i:
        url_columns=url_columns+[i]
        
print ('\n''url columns:\n\n', url_columns)
print ('\n''unique value columns:\n\n', unique_value_columns)

In [ ]:
warnings.filterwarnings("ignore")

relevant_columns=data['listings'].columns-url_columns-unique_value_columns
relevant_columns

data['listings']=data['listings'][relevant_columns]

for i in data['listings'].columns:
    
    if set(data['listings'][i])=={'t','f'}:
        data['listings'][i]=data['listings'][i].apply(to_binary)
        
data['listings']['price']=data['listings']['price'].map(lambda x: correction(x))

In [ ]:
data['listings'][relevant_columns].info()

In [ ]:
reviews=[]

for i in relevant_columns:
    if 'review_scores' in i:
        reviews=reviews+[i]

In [ ]:
PP_Data=data['listings'][data['listings']['number_of_reviews']!=0][reviews+['number_of_reviews','price']]
PP_Data=PP_Data.dropna()

def grading(x):
    
    percentage=[97,93,90,87,83,80,77,73,70,67,63,60,0]
    grade=['A+','A','A-','B+','B','B-','C+','C','C-','D+','D','D-','F']
    
    for i in np.arange(len(percentage)):
    
        if (x)>=percentage[i]:
            return (grade[i])
        
PP_Data['RSR_Letters']=PP_Data['review_scores_rating'].map(lambda x: grading(x))

In [ ]:
sns.pairplot(PP_Data[['review_scores_rating','number_of_reviews','price','RSR_Letters']], dropna=True, palette="RdBu", hue='RSR_Letters', hue_order=['A+','A','A-','B+','B','B-','C+','C','C-','D+','D','D-','F'])

Low price establishments tend to have lower review ratings. They also tend to do business with airb&b a smaller amount of times. <br>
While its expected to get less quality for a cheaper product, people's satisfication seems to be more related to the quality of the establishment rather them the cost-benefit relation.


In [ ]:
for i in ['review_scores_rating','number_of_reviews']:
    print (i, '\t',round(pearsonr(PP_Data[i], PP_Data['price'])[0],2))

In [ ]:
numerical=[]

for i in data['listings'].columns:
    if data['listings'][i].dtype=='int64':
        numerical=numerical+[i]
    if data['listings'][i].dtype=='float64':
        numerical=numerical+[i]

In [ ]:
A=data['listings'][numerical].corr().abs().unstack().order(kind='quicksort', ascending=False)
A=A[A!=1]

In [ ]:
A.loc['price'][:10]

We can identify the 3 most important factors that correlate to price, they are in order: <br>
    
How many ppl it accomodates: square_feet, accomodates, bedrooms, beds, bathrooms and guest_included are all related to how many people can be accomodated in the establishment. <br>

Location: latitude and  review scores location most likely relate to the house geographic location, since its all data taken from one location, i am assuming the northern area is supposed to be 'better'. <br>
    
Host's experience: listing count and total listing count relate to how experienced the host is.

In [ ]:
A.loc['review_scores_rating'][6:13] #first values were ommited because they were other review scores.

In [ ]:
A.loc['review_scores_value'][6:13] #first values were ommited because they were other review scores.

its unclear to me the difference between review score value and review score rating. <br>
Since theres no glossary explaining the features, i looked into both of em for correlations. <br>
<br>
The first values on the correlation are all other review scores. Which indicates that if a guest had an overall pleasent stay, hes more likelly to grade highly all atributes. <br>
Others value relate to the size of the establishment, its short notice avaialability and the host's experience.